# **Lab 1 Data Manipulation**

### **Data Extraction**

In [2]:
# Transform .dat file to .csv
import csv
for file in ["NACA0012_25Hz_70pitch", "NACA4412_25Hz_70pitch"]:
    datContent = []
    for i in open(file + ".dat").readlines():
        if str(i[0]) in ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "-"]:
            datContent.append(i.strip().split())

    with open(file + ".csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(datContent)

In [30]:
# Transform .csv to DataFrames
import pandas as pd

naca0012_data = {}
naca4412_data = {}

for file in ["NACA0012_25Hz_70pitch", "NACA4412_25Hz_70pitch"]:

    if file == "NACA0012_25Hz_70pitch":
        storage = naca0012_data
    elif file == "NACA4412_25Hz_70pitch":
        storage = naca4412_data

    df_composite = pd.read_csv(file + ".csv", index_col = 0, header = None, names = ["Dyn. Pressure (In. H20)", "Surf. Pressure (In. H20)"])

    chunk_size = 19
    num_chunks = int(len(df_composite.index) / 19)

    for i in range(num_chunks):
        chunk_old = df_composite.iloc[chunk_size * i:chunk_size * (i+1)]
        chunk = chunk_old.iloc[1:, :]
        chunk.name = int(chunk_old.index[0])

        storage[chunk.name] = chunk


In [ ]:
# Correcting pressure tap 17

# Placeholder for Gracynne's numbers

In [31]:
# Calculating U, Re, Cp

U_naca0012 = 33.3 # m/s
U_naca4412 = 34.0 # m/s

chord = .1524 # m [6 in]

p_atm = 98700 # Pa [29.15 in Hg]
T_atm = 300.93 # K [82 F, 27.78 C]
rho = 1.14486 # kg/m3 [at given p_atm and t_atm]
mu = 18.5 * 10**-6 # Pa*s

re_naca0012 = rho * U_naca0012 * chord / mu
re_naca4412 = rho * U_naca4412 * chord / mu

for item in naca0012_data:
    tbl = naca0012_data[item]
    tbl["Cp"] = ((pd.to_numeric(tbl["Surf. Pressure (In. H20)"]) * 249.08) - p_atm) / (pd.to_numeric(tbl["Dyn. Pressure (In. H20)"]) * 249.08)
    tbl["U"] = (2 * (pd.to_numeric(tbl["Dyn. Pressure (In. H20)"]) * 249.08) / rho) ** .5
    naca0012_data[item] = tbl

for item in naca4412_data:
    tbl = naca4412_data[item]
    tbl["Cp"] = ((pd.to_numeric(tbl["Surf. Pressure (In. H20)"]) * 249.08) - p_atm) / (pd.to_numeric(tbl["Dyn. Pressure (In. H20)"]) * 249.08)
    tbl["U"] = (2 * (pd.to_numeric(tbl["Dyn. Pressure (In. H20)"]) * 249.08) / rho) ** .5
    naca4412_data[item] = tbl

C:\Users\djebs\AppData\Local\Temp/ipykernel_17972/3854861553.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl["Cp"] = ((pd.to_numeric(tbl["Surf. Pressure (In. H20)"]) * 249.08) - p_atm) / (pd.to_numeric(tbl["Dyn. Pressure (In. H20)"]) * 249.08)
C:\Users\djebs\AppData\Local\Temp/ipykernel_17972/3854861553.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl["U"] = (2 * (pd.to_numeric(tbl["Dyn. Pressure (In. H20)"]) * 249.08) / rho) ** .5


In [35]:
naca0012_data[12]

,Dyn. Pressure (In. H20),Surf. Pressure (In. H20),Cp,U
1.0,2.553,0.727,-154.928018,33.329872
2.0,2.547,-0.688,-155.848540,33.290683
3.0,2.555,-0.754,-155.386392,33.342924
4.0,2.552,-0.792,-155.583946,33.323344
5.0,2.554,-0.792,-155.462111,33.336399
6.0,2.546,-0.805,-155.955707,33.284147
7.0,2.547,-0.803,-155.893691,33.290683
8.0,2.550,-0.795,-155.707149,33.310283
9.0,2.550,-0.796,-155.707541,33.310283
10.0,2.559,-0.788,-155.156792,33.369014


In [36]:
naca4412_data[12]

,Dyn. Pressure (In. H20),Surf. Pressure (In. H20),Cp,U
1.0,2.651,-3.755,-150.891449,33.963552
2.0,2.656,-5.552,-151.283972,33.995566
3.0,2.656,-5.025,-151.085554,33.995566
4.0,2.655,-3.167,-150.442648,33.989165
5.0,2.657,-2.545,-150.095307,34.001965
6.0,2.658,-1.953,-149.816114,34.008363
7.0,2.654,-1.370,-149.822242,33.982764
8.0,2.650,-0.810,-149.837068,33.957145
9.0,2.645,-0.226,-149.899520,33.925095
10.0,2.656,-0.187,-149.264017,33.995566


In [12]:
pd.to_numeric(naca0012_data[0]["Surf. Pressure (In. H20)"])

1.0     1.390
2.0    -1.056
3.0    -0.967
4.0    -0.878
5.0    -0.822
6.0    -0.771
7.0    -0.713
8.0    -0.638
9.0    -0.573
10.0   -0.510
11.0   -0.451
12.0   -0.401
13.0   -0.319
14.0   -0.203
15.0   -0.126
16.0   -0.065
17.0    0.007
18.0    0.099
Name: Surf. Pressure (In. H20), dtype: float64